In [1]:
import numpy as np
import pandas as pd
import sys
import os

In [2]:
%load_ext autoreload
%autoreload 2

In [3]:
os.getcwd()

'/Users/harshbaberwal/Desktop/Projects/git_repos/tattoo_style_tagger/development'

In [4]:
sys.path.append("/Users/harshbaberwal/Desktop/Projects/git_repos/tattoo_style_tagger")

In [5]:
style_info = pd.read_csv("../data/style_queries_w_description.csv")
style_info.head()

,style,style_query,style_desc
0,abstract,abstract,"Just like the art movement it is inspired by, ..."
1,anime,anime,NaN
2,black_and_gray,black_and_gray,Shading is key when it comes to black and grey...
3,blackwork,blackwork,Black and bold is what makes Blackwork tattoos...
4,chicano,chicano,Often using black and grey and an illustrative...


In [6]:
styles_for_test = style_info.head(3)
styles_for_test

,style,style_query,style_desc
0,abstract,abstract,"Just like the art movement it is inspired by, ..."
1,anime,anime,NaN
2,black_and_gray,black_and_gray,Shading is key when it comes to black and grey...


## Meta Data Pull

In [8]:
from lib.data_pull.meta_data_pull import read_tattoos_meta_data

In [9]:
all_styles_meta_data = read_tattoos_meta_data(styles_for_test)

Reading style abstract
Done reading meta data for style abstract 
Reading style anime
Done reading meta data for style anime 
Reading style black_and_gray
Done reading meta data for style black_and_gray 


In [10]:
all_styles_meta_data.head(3)

,tattoo_id,image_url,styles,searched_style_query
0,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,"['abstract', 'black_and_gray', 'blackwork', 'f...",abstract
1,1053382,https://d1kq2dqeox7x40.cloudfront.net/images/p...,"['abstract', 'black_and_gray', 'blackwork', 'h...",abstract
2,531019,https://d1kq2dqeox7x40.cloudfront.net/images/p...,"['abstract', 'ornamental', 'watercolor']",abstract


In [11]:
all_styles_meta_data.to_csv("temp/all_styles_meta_data.csv", index=False)

In [12]:
all_styles_meta_data = pd.read_csv("temp/all_styles_meta_data.csv")

## Meta Data Processing

## 1

In [15]:
from lib.data_pull.meta_data_processing import process_tattoo_meta_data_to_long_form

In [16]:
tattoo_meta_data_exploded = process_tattoo_meta_data_to_long_form(all_styles_meta_data)

In [17]:
tattoo_meta_data_exploded.head()

,tattoo_id,image_url,styles
0,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,abstract
0,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,black_and_gray
0,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,blackwork
0,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,fine_line
0,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,horror


#### Sanity Check

In [19]:
all_styles_meta_data.tattoo_id.nunique() - tattoo_meta_data_exploded.tattoo_id.nunique()

589

In [20]:
all_styles_meta_data[all_styles_meta_data['styles'] == "[]"].tattoo_id.nunique()

589

## 2

In [22]:
from lib.data_pull.meta_data_processing import filter_out_tattoos

In [23]:
tattoo_meta_data_exploded_filtered = filter_out_tattoos(tattoo_meta_data_exploded)

In [24]:
tattoo_meta_data_exploded_filtered.head()

,tattoo_id,image_url,styles
0,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,abstract
1,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,black_and_gray
2,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,blackwork
3,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,fine_line
4,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,horror


#### Sanity Check

In [26]:
tattoo_meta_data_exploded.tattoo_id.nunique() - tattoo_meta_data_exploded_filtered.tattoo_id.nunique()

122

In [27]:
tattoo_meta_data_exploded.groupby(
    'image_url', as_index=False
    )['styles'].nunique().query("styles > 7").shape

(122, 2)

## 3

In [29]:
from lib.data_pull.meta_data_processing import handle_duplicates

In [30]:
tattoo_meta_data_exploded_filtered_final = handle_duplicates(tattoo_meta_data_exploded_filtered)

orig_tattoo_id    0
image_url         0
styles            0
tattoo_id         0
dtype: int64


#### Sanity Check

In [32]:
tattoo_meta_data_exploded_filtered.image_url.nunique() - tattoo_meta_data_exploded_filtered_final.image_url.nunique()

0

In [33]:
tattoo_meta_data_exploded_filtered.tattoo_id.nunique() - tattoo_meta_data_exploded_filtered_final.tattoo_id.nunique()

0

In [34]:
tattoo_meta_data_exploded_filtered.groupby(
    'image_url', as_index=False
    )['tattoo_id'].nunique().query("tattoo_id > 1").shape

(0, 2)

In [35]:
tattoo_meta_data_for_testing = tattoo_meta_data_exploded_filtered_final.head(20)

## Tattoo Images Download and Image Processing

### Image Downloads

In [38]:
from lib.data_pull.download_tattoo_images import download_tattoo_images

In [39]:
download_path = "/Users/harshbaberwal/Desktop/Projects/git_repos/tattoo_style_tagger/development/temp/"

In [40]:
download_tattoo_images(
    tattoo_meta_data_for_testing,
    download_path = download_path
    )

In [41]:
raw_images_path = download_path+"raw_tattoo_images"

In [42]:
os.listdir(raw_images_path)

['4.jpg',
 '-1.jpg',
 '5.jpg',
 '6.jpg',
 '2.jpg',
 '-6.jpg',
 '3.jpg',
 '1.jpg',
 '-5.jpg']

In [43]:
tattoo_meta_data_for_testing.tattoo_id.unique()

array([1, 2, 3, 4, 5, 6])

### Image Processing

In [45]:
from lib.data_preprocessing.data_pre_processing import (
get_dims_for_all_images,
flag_images_with_dims_outliers,
filter_out_less_represented_styles
)


In [46]:
tattoos_dim_df, _ = get_dims_for_all_images(tattoo_meta_data_for_testing, download_path+"/raw_tattoo_images")

In [47]:
tattoos_dim_df

,tattoo_id,height,width
0,1,1515,1010
1,2,3993,2662
2,3,1334,750
3,4,782,782
4,5,2716,1811
5,6,3258,2171


In [48]:
tattoo_meta_data_for_testing_f = flag_images_with_dims_outliers(tattoo_meta_data_for_testing, tattoos_dim_df)

In [49]:
tattoo_meta_data_for_testing_f

,orig_tattoo_id,image_url,styles,tattoo_id,to_process
0,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,abstract,1,True
1,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,black_and_gray,1,True
2,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,blackwork,1,True
3,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,fine_line,1,True
4,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,horror,1,True
5,1011413,https://d1kq2dqeox7x40.cloudfront.net/images/p...,illustrative,1,True
6,1053382,https://d1kq2dqeox7x40.cloudfront.net/images/p...,abstract,2,False
7,1053382,https://d1kq2dqeox7x40.cloudfront.net/images/p...,black_and_gray,2,False
8,1053382,https://d1kq2dqeox7x40.cloudfront.net/images/p...,blackwork,2,False
9,1053382,https://d1kq2dqeox7x40.cloudfront.net/images/p...,horror,2,False


In [50]:
tattoo_meta_data_for_testing_ff = filter_out_less_represented_styles(tattoo_meta_data_for_testing_f)

In [51]:
tattoo_meta_data_for_testing_ff

,orig_tattoo_id,image_url,styles,tattoo_id,to_process


### Image Augmentation

In [53]:
tattoo_meta_data_for_testing_ff = tattoo_meta_data_for_testing_f

In [54]:
from lib.data_preprocessing.data_augmentation import (
prep_metadata_w_flipped_image_ids,
flip_images_to_augment_data
)


In [55]:
tattoo_meta_data_processed_augmented = prep_metadata_w_flipped_image_ids(tattoo_meta_data_for_testing_ff)

/Users/harshbaberwal/Desktop/Projects/git_repos/tattoo_style_tagger/lib/data_preprocessing/data_augmentation.py:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tattoo_meta_data_processed_to_augment['tattoo_id_flipped'] = \


In [56]:
tattoo_meta_data_processed_augmented

,tattoo_id,styles
0,1,abstract
1,1,black_and_gray
2,1,blackwork
3,1,fine_line
4,1,horror
5,1,illustrative
16,5,abstract
17,5,illustrative
18,5,trashpolka
19,6,abstract


In [57]:
raw_images_path

'/Users/harshbaberwal/Desktop/Projects/git_repos/tattoo_style_tagger/development/temp/raw_tattoo_images'

In [58]:
flip_images_to_augment_data(tattoo_meta_data_processed_augmented,raw_images_path)

In [59]:
os.listdir(raw_images_path)

['4.jpg',
 '-1.jpg',
 '5.jpg',
 '6.jpg',
 '2.jpg',
 '-6.jpg',
 '3.jpg',
 '1.jpg',
 '-5.jpg']